In [1]:
import pandas as pd

In [2]:
books = pd.read_csv('book_cleaned.csv')

In [5]:
books['categories'].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Aged women,1
475,Imperialism,1
476,Human-animal relationships,1
477,Amish,1


In [8]:
books['categories'].value_counts().reset_index().query('count > 50')

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [9]:
category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Children's Fiction",
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Children's Nonfiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction"}

books["simple_categories"] = books["categories"].map(category_mapping)

In [10]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title and subtitle,taggged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN


In [11]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

c:\Users\Reyan\Desktop\Projects\Book Recommender\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Reyan\Desktop\Projects\Book Recommender\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Reyan\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an a

In [19]:
sequence = books.loc[books["simple_categories"]=='Fiction','description'].reset_index(drop=True)[0]
ficition_categories = ['Fiction','Nonfiction']


In [20]:
classifier(sequence, ficition_categories)


{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [21]:
import numpy as np

max_index = np.argmax(classifier(sequence, ficition_categories)['scores'])
max_label = classifier(sequence, ficition_categories)['labels'][max_index]
max_label


'Fiction'

In [22]:
def generate_prediction(sequence, categories):
    prediction = classifier(sequence, categories)
    max_index = np.argmax(prediction['scores'])
    max_label = prediction['labels'][max_index]
    return max_label


In [26]:
from tqdm import tqdm

actual_categories = []
predicted_categories = []

for index in tqdm(range(0,300)):
    sequence = books.loc[books["simple_categories"]=='Fiction','description'].reset_index(drop=True)[index]
    actual_categories.append("Fiction")
    predicted_categories.append(generate_prediction(sequence, ficition_categories))

for index in tqdm(range(0,300)):
    sequence = books.loc[books["simple_categories"]=='Nonfiction','description'].reset_index(drop=True)[index]
    actual_categories.append("Nonfiction")
    predicted_categories.append(generate_prediction(sequence, ficition_categories))





100%|██████████| 300/300 [04:30<00:00,  1.11it/s]


In [27]:
prediction_df = pd.DataFrame({"actual_category": actual_categories, "predicted_category": predicted_categories})


In [29]:
prediction_df.head(10)


,actual_category,predicted_category
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction
5,Fiction,Fiction
6,Fiction,Fiction
7,Fiction,Fiction
8,Fiction,Fiction
9,Fiction,Fiction


In [30]:
prediction_df['correct_prediction'] = (prediction_df['actual_category'] == prediction_df['predicted_category']).astype(int)


In [31]:
prediction_df.head(10)

,actual_category,predicted_category,correct_prediction
0,Fiction,Fiction,1
1,Fiction,Fiction,1
2,Fiction,Fiction,1
3,Fiction,Nonfiction,0
4,Fiction,Fiction,1
5,Fiction,Fiction,1
6,Fiction,Fiction,1
7,Fiction,Fiction,1
8,Fiction,Fiction,1
9,Fiction,Fiction,1


In [37]:
prediction_df['correct_prediction'].value_counts()[1] / len(prediction_df)


np.float64(0.7783333333333333)

In [38]:
isbn=[]
prediction_cat=[]

missing_cat = books.loc[books['simple_categories'].isnull(),['isbn13','description']].reset_index(drop=True)



In [40]:
for i in tqdm(range(0,len(missing_cat))):
    isbn.append(missing_cat.loc[i,'isbn13'])
    prediction_cat.append(generate_prediction(missing_cat.loc[i,'description'],ficition_categories))



100%|██████████| 1454/1454 [20:50<00:00,  1.16it/s]


In [47]:
missing_prediction_df = pd.DataFrame({"isbn13":isbn,"prediction_cat":prediction_cat})


In [48]:
missing_prediction_df

,isbn13,prediction_cat
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1449,9788125026600,Nonfiction
1450,9788171565641,Fiction
1451,9788172235222,Fiction
1452,9788173031014,Nonfiction


In [49]:
books = pd.merge(books,missing_prediction_df,on='isbn13',how='left')

books['simple_categories'] = np.where(books['simple_categories'].isnull(),books['prediction_cat'],books['simple_categories'])



In [50]:
books = books.drop(columns=['prediction_cat'])

books.head()


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title and subtitle,taggged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction


In [51]:
books.isnull().sum()

isbn13                   0
isbn10                   0
title                    0
authors                 32
categories              30
thumbnail              166
description              0
published_year           0
average_rating           0
num_pages                0
ratings_count            0
title and subtitle       0
taggged_description      0
simple_categories        0
dtype: int64

In [52]:
books.to_csv('book_with_categories.csv',index=False)